In [1]:
#https://www.machinelearningplus.com/nlp/gensim-tutorial/

### 5. How to create a bag of words corpus in gensim?

The next important object you need to familiarize with in order to work in gensim is the Corpus (a Bag of Words). That is, it is a corpus object that contains the word id and its frequency in each document. You can think of it as gensim’s equivalent of a Document-Term matrix.

Once you have the updated dictionary, all you need to do to create a bag of words corpus is to pass the tokenized list of words to the Dictionary.doc2bow()

Let’s create s Corpus for a simple list (my_docs) containing 2 sentences.

In [17]:
import gensim
from gensim import corpora
from pprint import pprint
from gensim.utils import simple_preprocess
from smart_open import smart_open
import os


# List with 2 sentences
my_docs = ["Who let the dogs out?",
           "Who? Who? Who? Who?"]

# Tokenize the docs
# tokenized_list2 = [[text for text in doc.split()] for doc in my_docs]  simple_preprocess is preferred as it lowercases
tokenized_list = [simple_preprocess(doc) for doc in my_docs]


# Create the Corpus
mydict = corpora.Dictionary()
mycorpus = [mydict.doc2bow(doc, allow_update=True) for doc in tokenized_list]

pprint(mydict.token2id)  #print token to id  
pprint(mycorpus)    #print BoW
#> [[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)], [(4, 4)]]

{'dogs': 0, 'let': 1, 'out': 2, 'the': 3, 'who': 4}
[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)], [(4, 4)]]


In [18]:
#or more efficient
word_counts = [[(mydict[id], count) for id, count in line] for line in mycorpus]
#get the id and count from every line in mycorpus, then fill in that id in mydict to get the token and print the same count
pprint(word_counts)

[[('dogs', 1), ('let', 1), ('out', 1), ('the', 1), ('who', 1)], [('who', 4)]]


#### Notice, the order of the words gets lost. Just the word and it’s frequency information is retained

# 6. How to create a bag of words corpus from a text file?

In [21]:
from gensim.utils import simple_preprocess
from smart_open import smart_open
import nltk


class BoWCorpus(object):
    def __init__(self, path, dictionary):
        self.filepath = path
        self.dictionary = dictionary

    def __iter__(self):
        global mydict  # OPTIONAL, only if updating the source dictionary.
        for line in smart_open(self.filepath, encoding='latin'):
            # tokenize
            tokenized_list = simple_preprocess(line, deacc=True)

            # create bag of words
            bow = self.dictionary.doc2bow(tokenized_list, allow_update=True)

            # update the source dictionary (OPTIONAL)
            mydict.merge_with(self.dictionary)

            # lazy return the BoW
            yield bow


# Create the Dictionary
mydict = corpora.Dictionary()

# Create the Corpus
bow_corpus = BoWCorpus('sample.txt', dictionary=mydict)  # memory friendly

# Print the token_id and count for each line.
for line in bow_corpus:
    print(line)

print([[(mydict[id], count) for id, count in line] for line in bow_corpus])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1)]
[(14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1)]
[(5, 2), (12, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1)]
[(3, 1), (9, 1), (12, 2), (18, 1), (22, 1), (26, 1), (32, 1), (33, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1)]
[(15, 1), (17, 1), (18, 1), (21, 1)]
[(3, 1), (9, 1), (14, 1), (16, 1), (19, 1), (22, 2), (26, 2), (32, 1), (33, 1), (34, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1)]
[(3, 1), (5, 2), (9, 1), (10, 1), (12, 1), (13, 1), (18, 1), (43, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1)]
[(12, 1), (22, 1), (33, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1)]
[(12, 3), (16, 1), (26, 1), (41, 1), (43, 1), (47, 1), (66, 1), (67, 1), (68, 1), (69, 1), (

### 7. How to save a gensim dictionary and corpus to disk and load them back? 

In [22]:
# Save the Dict and Corpus
mydict.save('mydict.dict')  # save dict to disk
corpora.MmCorpus.serialize('bow_corpus.mm', bow_corpus)  # save corpus to disk

In [23]:
# Load them back
loaded_dict = corpora.Dictionary.load('mydict.dict')

corpus = corpora.MmCorpus('bow_corpus.mm')
for line in corpus:
    print(line)

[(0, 1.0), (1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0), (6, 1.0), (7, 1.0), (8, 1.0), (9, 1.0), (10, 1.0), (11, 1.0), (12, 1.0), (13, 1.0)]
[(14, 1.0), (15, 1.0), (16, 1.0), (17, 1.0), (18, 1.0), (19, 1.0), (20, 1.0), (21, 1.0)]
[(5, 2.0), (12, 1.0), (22, 2.0), (23, 1.0), (24, 1.0), (25, 1.0), (26, 1.0), (27, 1.0), (28, 1.0), (29, 1.0), (30, 1.0), (31, 1.0), (32, 1.0), (33, 1.0), (34, 1.0)]
[(3, 1.0), (9, 1.0), (12, 2.0), (18, 1.0), (22, 1.0), (26, 1.0), (32, 1.0), (33, 1.0), (35, 1.0), (36, 1.0), (37, 1.0), (38, 1.0), (39, 1.0), (40, 1.0), (41, 1.0), (42, 1.0)]
[(15, 1.0), (17, 1.0), (18, 1.0), (21, 1.0)]
[(3, 1.0), (9, 1.0), (14, 1.0), (16, 1.0), (19, 1.0), (22, 2.0), (26, 2.0), (32, 1.0), (33, 1.0), (34, 1.0), (43, 1.0), (44, 1.0), (45, 1.0), (46, 1.0), (47, 1.0)]
[(3, 1.0), (5, 2.0), (9, 1.0), (10, 1.0), (12, 1.0), (13, 1.0), (18, 1.0), (43, 1.0), (47, 1.0), (48, 1.0), (49, 1.0), (50, 1.0), (51, 1.0), (52, 1.0), (53, 1.0), (54, 1.0), (55, 1.0), (56, 1.0), (57, 1.0)]
[(12, 1.0)